In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


True
1
NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [1]:
import gradio as gr
import torch
import torch.nn as nn
from PIL import Image
from facenet_pytorch import MTCNN
from torchvision import transforms, models

# ------------------------------
# DEVICE (GPU if available)
# ------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ------------------------------
# MTCNN FACE DETECTOR (SINGLE FACE, GPU)
# ------------------------------
# smaller detection size for speed
mtcnn = MTCNN(keep_all=False, device=device, image_size=160, margin=20)

def detect_and_crop_face(image: Image.Image, padding_ratio=0.15):
    """
    Detect a single face with MTCNN and crop it.
    """
    # Resize image smaller for faster detection
    small_image = image.resize((320, 320))
    boxes, probs = mtcnn.detect(small_image)

    if boxes is None or len(boxes) == 0:
        return None

    # Only one box because keep_all=False
    x1, y1, x2, y2 = boxes[0]

    # Scale back to original image
    scale_x = image.width / 320
    scale_y = image.height / 320
    x1, y1, x2, y2 = x1 * scale_x, y1 * scale_y, x2 * scale_x, y2 * scale_y

    # Add padding
    w, h = x2 - x1, y2 - y1
    pad = padding_ratio * max(w, h)
    x1_p = max(0, int(x1 - pad))
    y1_p = max(0, int(y1 - pad))
    x2_p = min(image.width, int(x2 + pad))
    y2_p = min(image.height, int(y2 + pad))

    # Crop face
    face = image.crop((x1_p, y1_p, x2_p, y2_p))
    return face

# ------------------------------
# LOAD EFFICIENTNET AGE MODEL
# ------------------------------
weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1
model_ft = models.efficientnet_b0(weights=weights)

# Freeze parameters except last few layers
for param in model_ft.parameters():
    param.requires_grad = False
for param in model_ft.features[-4:].parameters():
    param.requires_grad = True

in_features = model_ft.classifier[1].in_features
model_ft.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 5)
)

# Load trained model
state_dict = torch.load("C:/Users/ROG/Vision/outputs/B0/best_model_64.pt", map_location=device)
model_ft.load_state_dict(state_dict)
model_ft.to(device)
model_ft.eval()

# Half precision for GPU speed-up
if device == "cuda":
    model_ft.half()

print("🚀 EfficientNet model loaded and ready!")

# ------------------------------
# IMAGE TRANSFORM
# ------------------------------
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

age_groups = ['Người lớn tuổi', 'Thanh niên', 'Thiếu niên', 'Trung niên', 'Trẻ em']

# ------------------------------
# PREDICTION FUNCTION
# ------------------------------
def predict(image: Image.Image):
    face = detect_and_crop_face(image)
    if face is None:
        raise gr.Error("❌ Không tìm thấy khuôn mặt. Hãy chọn ảnh khác!")

    # Transform and move to device
    img = transform(face).unsqueeze(0).to(device)
    if device == "cuda":
        img = img.half()  # match model precision

    # Inference
    with torch.no_grad():
        logits = model_ft(img)
        probs = torch.softmax(logits, dim=1)[0]

    return face, {age_groups[i]: float(probs[i]) for i in range(5)}

# ------------------------------
# GRADIO INTERFACE
# ------------------------------
gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=[
        gr.Image(type="pil", label="Cropped Face"),
        gr.Label(num_top_classes=5, label="Age Prediction")
    ],
    title="Age Classifier – MTCNN + EfficientNet (Optimized)",
    description="Detect only faces using MTCNN (GPU if available), crop them, and predict age group using EfficientNet."
).launch(share=True, inbrowser=True)


C:\Users\ROG\Vision\venv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
🚀 EfficientNet model loaded and ready!
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
